<!---
 <IMG SRC=\"https://github.com/jacquesroy/byte-size-data-science/raw/master/images/Banner.png\" ALT=\"BSDS Banner\" WIDTH=1195 HEIGHT=200>
 -->
![Banner](./images/Banner.png)

### Simple example on the TensorFlow [home page](https://www.tensorflow.org/)

In [ ]:
# Import the libraries used in this notebook
import matplotlib.pyplot as plt
from tensorflow import keras
import pandas as pd
import numpy as np

### Load the MNIST dataset
Keep a version of the original integer arrays before making them floats

Each image is a 28x28 array which gives an input of 784 values

In [ ]:
# Read the dataset
mnist = keras.datasets.mnist

(x_train0, y_train),(x_test0, y_test) = mnist.load_data()
# The 28x28 arrays are integers between 0 and 255
# Convert to real numbers between 0 and 1
x_train, x_test = x_train0 / 255.0, x_test0 / 255.0

print("Size of the training set: {}".format(x_train.shape))
print("Size of the testing  set: {}".format(x_test.shape))

### Create the model
1. input layer: Flatten the array into a vector
2. Create a densely connected hidden layer with 128 nodes and using a ReLU activation function
3. Set a dropout of 20%. This means that 20% of the values are set to 0 randomly to make sure the model does not depend on specific nodes too strongly.
4. Connect the hidden layer to 10 output value using a softmax activation function

![Neural Network high-level diagram](./images/NN_MNISTDiagram.png)

In [ ]:
# Create a model
#
# UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, 
# prefer using an `Input(shape)` object as the first layer in the model instead.
#
model = keras.models.Sequential([
  keras.layers.Flatten(input_shape=(28, 28)), # Flattens returns 784 input values
  # keras.layers.Input(shape=(28, 28) ), 
  keras.layers.Dense(128, activation='relu'), #, input_shape=(28x28)),
  keras.layers.Dropout(0.2),
  keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy'])

In [ ]:
# Display an numerical representation of a training digit
# Better display when using the integers instead of the float values
df = pd.DataFrame(x_train0[0])
print(df.to_string(index=False, header=False))

In [ ]:
# Display the image itself, and the value it represents, using matplotlib
plt.imshow(x_train[0], cmap='gray')
plt.title('Label: ' + str(y_train[0]))
plt.show()

In [ ]:
# Display some information about the model (before training)
print(f"Model input shape: {model.input_shape}")
print(f"Number of layers: {len(model.layers)}")
print(f"Layer names: " + ", ".join([type(layer).__name__ for layer in model.layers]))

model.summary()

In [ ]:
w_list = model.get_weights()
w_list[0][:20,0:1].flatten()

In [ ]:
# Display some model weights
layer_names = [type(layer).__name__ for layer in model.layers]
layer_names.append("output")
# print(layer_names)
w_list = model.get_weights()
nb_layers = len(w_list)
print(f"weights matrix length (number of layers): {nb_layers}")
for ix in range(nb_layers):
    print(f"Layer \033[1;32m{layer_names[ix]} to {layer_names[ix+1]}\033[0m shape: {w_list[ix].shape}")
    # print(f"Layer {ix} shape: {w_list[ix].shape}")
print("\nFirst 20 pixels (of 784) initial weights for the first neuron of the hidden layer:")
print(w_list[0][:20,0:1].flatten()) # get list[0], get the 784 values form the first neuron

print("\nFirst 20 neurons (of 128) initial weights for the first pixel going to each neuron of the hidden layer:")
print(w_list[0][0:1, :20].flatten()) # get list[0], get the 784 values form the first neuron


In [ ]:
# Train the model
# Each epoch processes the entire dataset in batches. Default batch size is 32
# FYI 1875 batches of 32 is 60,000 values
model.fit(x_train, y_train, epochs=5)

In [ ]:
# Test the model. Metrics: 'loss', 'accuracy'
model.evaluate(x_test, y_test)

In [ ]:
predictions = model.predict(x_test)

In [ ]:
print(predictions[0])
# Show the 10 output values for the first test digit

In [ ]:
# Change the precision to make it more human-readable
np.set_printoptions(formatter={'float': lambda x: "{:.4f}".format(x)})
print("  0      1      2      3      4      5      6      7      8      9")
print(predictions[0])
np.set_printoptions(formatter=None)

In [ ]:
# max(predictions[0])
# type(np.where( predictions[0] == max(predictions[0]) ))
for ix in range(0, 10) :
    prediction = np.where( predictions[ix] == max(predictions[ix]) )[0][0].item()
    expected = y_test[ix].item()
    if expected == prediction :
        print(f"\033[32mPrediction: {prediction}, expected: {expected}\033[0m")
    else :
        print(f"\033[31mPrediction: {prediction}, expected: {expected}\033[0m")

In [ ]:
keras.utils.plot_model(
    model, to_file='./model.png', show_shapes=True, show_dtype=False,
    show_layer_names=False, rankdir='TB', expand_nested=True, dpi=96,
    layer_range=None
)